In [1]:
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

import numpy as np

Using TensorFlow backend.


In [2]:
##Data Aquisition
me_test = np.load('MG_3jets_500000.npy', allow_pickle=True) #Matrix elements - test
momentum_test_r = np.load('LO_3_0.01/PS3_0.01_500000.npy', allow_pickle=True, encoding='bytes')#4-momenta of inputs - test

me_train = np.load('MG_3jets_3000000.npy', allow_pickle=True) #Matrix elements - train
momentum_train_r = np.load('LO_3_0.01/PS3_0.01_3000000.npy', allow_pickle=True, encoding='bytes') #4-momenta of input - train

In [3]:
##Reformat Momentum
momentum_train = np.array([np.ndarray.flatten(np.array(element)[3:, 2:]) for element in momentum_train_r])
momentum_test = np.array([np.ndarray.flatten(np.array(element)[3:, 2:]) for element in momentum_test_r])
##Rescale
scaler = StandardScaler()

momentum_train = scaler.fit_transform(momentum_train) #Rescale on training set
momentum_test = scaler.transform(momentum_test) #Rescale on test set

me_train = me_train*10**5
me_test = me_test*10**5

In [4]:
# Random Model from Tensorflow Website
def build_model(dropout, hidden_layers, optimizer='RMSprop'):
    
  model = keras.Sequential([
    keras.layers.Dense(10, activation='relu', input_shape=[4]), #could try swish??
    keras.layers.BatchNormalization(),  
    keras.layers.Dropout(dropout),
      
    *np.repeat([keras.layers.Dense(10, activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout)], hidden_layers),
    
    keras.layers.Dense(10, activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout),
    
    keras.layers.Dense(1) 
  ])

  model.compile(loss='mse', #gans?
                optimizer=optimizer,
                metrics=['mse'])
  return model

In [ ]:
model = KerasRegressor(build_fn=build_model, epochs=1)

dropout = [0.1, 0.2, 0.3]
hidden_layers = [5, 7, 9]
param_grid = dict(dropout=dropout, hidden_layers=hidden_layers)

grid = RandomizedSearchCV(model, param_grid, cv=3, scoring='neg_mean_absolute_error')
grid_result = grid.fit(momentum_train[:10000], me_train[:10000])

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [5]:
#model = build_model(grid_result.best_params_['dropout'],grid_result.best_params_['hidden_layers'])
model = build_model(0.5, 2)
model.fit(momentum_train, me_train, epochs=1)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/1
3000000/3000000 [==============================]3000000/3000000 [==============================] - 214s 71us/step - loss: 9.7719 - mean_squared_error: 9.7719



In [6]:
model.evaluate(momentum_test, me_test, batch_size=32, verbose=1)

500000/500000 [==============================]500000/500000 [==============================] - 13s 25us/step



[8.787978788966537, 8.787978788966537]

In [7]:
print(np.mean(me_train))
print(np.std(me_train))

0.5811934685242724
3.185513571309742


In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
print(mean_absolute_percentage_error(me_test, model.predict(momentum_test)))

In [9]:
print(model.predict(momentum_test[:10]))
print(me_test[:10])

[[0.42182222]
 [0.363656  ]
 [0.13414937]
 [0.40451664]
 [0.553161  ]
 [0.19603121]
 [0.2631011 ]
 [0.12869662]
 [0.25812036]
 [0.39833415]]
[0.06036061 0.06346955 0.14260086 0.0690008  5.57815624 0.02971246
 0.07641449 0.09079059 0.08664685 0.01896705]
